# Рекомендация тарифов

## ОГЛАВЛЕНИЕ

1. ПОСТАНОВКА ЗАДАЧИ
2. Открытие и изучение общей информации
3. Разбиение данных на выборки
4. Исследование моделей машинного обучения
* 4.1  Решающее дерево
* 4.2  Случайный лес
* 4.3  Логистическая регрессия
5. Проверка модели на тестовой выборке
6. Проверка модели на адекватность
7. Общий вывод по проекту

## 1 ПОСТАНОВКА ЗАДАЧИ

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* *сalls* — количество звонков,
* *minutes* — суммарная длительность звонков в минутах,
* *messages* — количество sms-сообщений,
* *mb_used* — израсходованный интернет-трафик в Мб,
* *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 2 Открытие и изучение общей информации

In [1]:
import pandas as pd
from IPython.display import display #импортируем метод дисплей для красивого вывода таблиц (не применяя print)
pd.set_option('display.max_columns', None) #реализуем возможность вывода всех столбцов на экран
from sklearn.tree import DecisionTreeClassifier #решающее дерево
from sklearn.model_selection import train_test_split # деление на 2 выборки
from sklearn.ensemble import RandomForestClassifier # случайный лес
from sklearn.linear_model import LogisticRegression # логистическая регрессия
from sklearn.metrics import accuracy_score # проверка качества

In [3]:
df = pd.read_csv('users_behavior.csv', sep=',')

print(df.info())

display(df.head())

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [4]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

*Из изучения датасета видим, что всего в нем 3214 строк и 5 столбцов. Пропущенные данные отсутствуют. Метод describe показал, что данные уже обработаны, выбросов практически нет, т.к. для всех 4-х параметров средние значения близки к медианным. На тарифе "Ультра" у нас 985 пользователей, остальные на "Смарте". По условиям задачи предобработка данных не требуется, можно переходить к разбиению данных на выборки.*

## 3 Разбиение данных на выборки

Всю нашу выборку надо разбить на обучающую, валидационную и тестовые выборки. Разобьем их в процентном соотношении 60/20/20. Для этого применим 2 раза метод train_test_split. В первой итерации отделим обучающую выборку от валидационной и тестовой, во второй разделим валидационную и тестовую.

In [5]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345) # разделили выборки в пропорции 4 к 10
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345) # разделили вал-ую и тестовую пополам

# проверим получившиеся размеры выборок
print(df_train.shape)
print(df_valid.shape)
print(df_test.shape)

(1928, 5)
(643, 5)
(643, 5)


1928 / 3214 ≈ 60 %, 643 / 3214 ≈ 20 %. Разбиение проведено корректно. 

## 4 Исследование моделей машинного обучения

Исследуем 3 модели машинного обучения для задачи классификации (решающее дерево, случайный лес и логистическая регрессия) и выберем лучшее для дальнейшей проверки на тестовой выборке. Перед переходом к примененю модели выделим обычные признаки и целевой признак обучающей и валидационной выборок.

In [6]:
train_features = df_train.drop(['is_ultra'], axis=1) # создаем признаки для обучающей выборки
train_target = df_train['is_ultra'] # создаем целевой признак для обучающей выборки
valid_features = df_valid.drop(['is_ultra'], axis=1) # создаем признаки для валидационной выборки
valid_target = df_valid['is_ultra'] # создаем целевой признак для валидационной выборки

### 4.1 Решающее дерево

In [7]:
%%time
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # создаем модель с заданной глубиной дерева
    model.fit(train_features, train_target) # обучаем модель на обучающей выборке
    predictions = model.predict(valid_features) # получим предсказания модели на валидационной выборке
    result = accuracy_score(valid_target, predictions) # посчитаем качество модели
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print('Наилучшее качество модели {:.3f} при глубине дерева {}'.format(best_result, best_depth))

Наилучшее качество модели 0.785 при глубине дерева 3
Wall time: 72 ms


### 4.2 Случайный лес

In [8]:
%%time
best_model = None
best_result = 0
best_est = 0
best_depth = 0
best_split = 0
for est in range(10, 51, 10):
    for depth in range (1, 9):
        for spl in (2, 5):
            model = RandomForestClassifier(random_state=12345, 
                                     n_estimators=est, 
                                     max_depth=depth, min_samples_split = spl) # инициализируем модель
            model.fit(train_features, train_target) # обучим модель на тренировочной выборке
            result = model.score(valid_features, valid_target) # посчитаем качество модели на валидационной выборке
            if result > best_result:
                best_model = model
                best_result = result
                best_est = est
                best_depth = depth
                best_split = spl

print("Accuracy наилучшей модели на валидационной выборке:", best_result, 
      "Количество деревьев:", best_est, "Максимальная глубина:", best_depth,
     "Минимальное количество примеров для разделения", best_split)

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 8 Минимальное количество примеров для разделения 2
Wall time: 5.78 s


### 4.3 Логистическая регрессия

In [9]:
%%time
model = LogisticRegression(random_state=12345) # инициализируем модель логистической регрессии с параметром random_state=12345
model.fit(train_features, train_target) # обучим модель на тренировочной выборке
result = model.score(valid_features, valid_target) # получим метрику качества модели на валидационной выборке

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7107309486780715
Wall time: 267 ms


Итак, мы видим, что наилучшее качество модели (0,809) на валидационной выборке получено при применении алгоритма "Случайный лес", чуть хуже у "Решающего дерева" - 0.785; и в конце списка "Логистическая регрессия" - 0.758. Проектом установлена метрика качества равная 0.75., значит любая модель может быть принята для проверки на тестовой выборке. Выберем модель с наилучшим качеством, т.е. "случайный лес" с определенными оптимальными параметрами: количество деревьев 40, глубина 7, минимальное количество примеров для разделения 2.

## 5 Проверка модели на тестовой выборке

In [10]:
test_features = df_test.drop(['is_ultra'], axis=1) # создаем признаки для тестовой выборки
test_target = df_test['is_ultra'] # создаем целевой признак для тестовой выборки

model_forest = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=7, min_samples_split = 2) # инициализируем модель
model_forest.fit(train_features, train_target) # обучим модель на тренировочной выборке
forest_test_result = model_forest.score(test_features, test_target) # посчитаем качество модели на тестовой выборке
print("Accuracy модели случайного леса на тестовой выборке:", forest_test_result)

Accuracy модели случайного леса на тестовой выборке: 0.80248833592535


Получили достаточно высокое качество (0,803) на тестовой выборке, что подтверждает работоспособность выбранного алгоритма и его праметров.

## 6 Проверка модели на адекватность

Параметр *Accuracy* обладает одним существенным недостатком, - его можно получить случайно. К примеру, если в выборке 950 отрицательных и 50 положительных объектов, то при абсолютно случайной классификации мы получим долю правильных ответов 0.95. Это означает, что доля положительных ответов сама по себе не несет никакой информации о качестве работы алгоритма, и вместе с ней следует анализировать и другие метрики. 

Перед переходом к самим метрикам необходимо ввести важную концепцию для описания этих метрик в терминах ошибок классификации — *confusion matrix* (матрица ошибок). 
- Если алгоритм предсказал "1" и это оказалось верно, то такое событие называют True Positive (верное положительное) или TP. 
- Если алгоритм предсказал "1" и это оказалось неверно, то такое событие называют False Positive (ложное положительное) или FP.
- Если алгоритм предсказал "0" и это оказалось верно, то такое событие называют True Negative (верное отрицательное) или TN. 
- Если алгоритм предсказал "0" и это оказалось неверно, то такое событие называют False Negative (ложное отрицательное) или FN.

Значение Accuracy в формате матрицы ошибок считается по следующей формуле:

$$Accuracy = \frac{TP+TN}{TP+TN+FP+FN}$$

Как было сказано выше, метрика accuracy может быть бесполнезна в задачах с неравными классами. Для оценки качества работы алгоритма на каждом из классов по отдельности введем метрики *precision* (точность) и *recall* (полнота).

$$Presision = \frac{TP}{TP+FP}$$

$$Recall = \frac{TP}{TP+FN}$$

Точность показывает, какая доля объектов, выделенных классификатором как положительные, действительно является положительными.

Полнота показывает, какая часть положительных объектов была выделена классификатором.

Для оценки взаимного влияния указаных метрик используется объединяющая их метрика - F-мера, которая является средним гармоническим precision и recall. F-мера вычисляется по формуле:

$$F = (1+\beta^2)*\frac{presision*recall}{(\beta^2*presision)+recall}$$

$\beta$ в данном случае определяет вес точности в метрике.

F-мера достигает максимума при полноте и точности, равными единице, и близка к нулю, если один из аргументов близок к нулю.

В sklearn есть удобная функция **_metrics.classificationreport**, возвращающая *recall*, *precision* и *F-меру* для каждого из классов, а также количество экземпляров каждого класса. Вызовем эту функцию для полученной модели.

In [11]:
from sklearn.metrics import classification_report # импортируем функцию отчета по классификации
report = classification_report(test_target, model_forest.predict(test_features), target_names=['Smart', 'Ultra'])
print(report)

              precision    recall  f1-score   support

       Smart       0.81      0.94      0.87       440
       Ultra       0.79      0.51      0.62       203

    accuracy                           0.80       643
   macro avg       0.80      0.72      0.74       643
weighted avg       0.80      0.80      0.79       643



Итак, мы видим, что выбранная модель более адекватно производит предсказания класса "0", который соответствует тарифу "Смарт". Все 3 метрики для данного класса на достаточно хорошем уровне. Менее адекватно модель производит предсказания для класса "1", которые соответствует тарифу "Ультра". На снижение адекватности повлияла метрика recall, которая для данного класса приняла значение 0.51.

## 7 Общий вывод по проекту

В ходе проверки трех различных алгоритмов машинного обучения для поставленной задачи классификации наилучшим оказался алгоритм "случайный лес" со следующими параметрами: количество деревьев 40, глубина 7, минимальное количество примеров для разделения 2. 
    
Данный алгоритм позволил получить значительную долю верных ответов как на валидационной (0,805), так и на тестовой (0,803) выборках. Оценка адекватности примененной модели показала, что наиболее лучшим образом модель справляется с предсказанием данных класса "0" (*precision = 0.81, recall = 0.94, f1 = 0.87*) и хуже с предсказанием данных класса "1" (*precision = 0.79, recall = 0.51, f1 = 0.62*).
    